In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../data/amazon_reviews_full.csv', index_col=0)

/home/reza/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
df.head()

,review_headline,review_body,star_rating
0,Quirky,Elmore Leonard meets the cast of Sierra Madre....,4
1,The Woman Who Wasn't There,This book was very interesting. It is a true s...,4
2,This Sleepy Sheep rocks!,I had the opportunity to review Mary had a Sle...,5
3,Steamy and suspenseful!!!!!,What a great read! I really couldn't put this...,5
4,Barbarians,Barbarians need love too ! Short stories work...,5


In [4]:
df['review'] = df['review_headline'] + '. ' + df['review_body']

In [5]:
df['review_sentiments'] = 0

In [6]:
df.loc[df['star_rating'].isin([4, 5]), 'review_sentiments'] = 1

In [7]:
df = df.drop(columns=['review_headline', 'review_body', 'star_rating'])

In [8]:
df.head()

,review,review_sentiments
0,Quirky. Elmore Leonard meets the cast of Sierr...,1
1,The Woman Who Wasn't There. This book was very...,1
2,This Sleepy Sheep rocks!. I had the opportunit...,1
3,Steamy and suspenseful!!!!!. What a great read...,1
4,Barbarians. Barbarians need love too ! Short ...,1


In [9]:
df['review_sentiments'].value_counts()

1    4128546
0     971663
Name: review_sentiments, dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5100209 entries, 0 to 5100208
Data columns (total 2 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   review             object
 1   review_sentiments  int64 
dtypes: int64(1), object(1)
memory usage: 77.8+ MB


In [11]:
def stratified_sample_df(df, col, n_samples):
    n = min(n_samples, df[col].value_counts().min())
    df_ = df.groupby(col).apply(lambda x: x.sample(n))
    df_.index = df_.index.droplevel(0)
    return df_

In [12]:
df_stratified = stratified_sample_df(df, 'review_sentiments', 500000)

In [13]:
df_stratified

,review,review_sentiments
4977479,No Longer A Fan of Amazon. I am completely dis...,0
4223645,Fifty Shades of Grey. Life is too short to rea...,0
291284,"A fun, quick read. A story about chasing super...",0
514476,Lost in execution. The author kindly provided ...,0
1452968,Self Help versus Bio. I bought this book becau...,0
...,...,...
3698072,Lonely on the Mountain. I have read a lot of \...,1
2684478,A Fun & Clever Read. A fun & clever read about...,1
2452699,"Love,Aubrey. There was a lot of \\""getting sic...",1
3975206,"Great steamy read, a lot of fun.. I really enj...",1


In [14]:
df_remaining = df.drop(labels=df_stratified.index, axis=0)

In [15]:
df_remaining

,review,review_sentiments
0,Quirky. Elmore Leonard meets the cast of Sierr...,1
1,The Woman Who Wasn't There. This book was very...,1
2,This Sleepy Sheep rocks!. I had the opportunit...,1
3,Steamy and suspenseful!!!!!. What a great read...,1
5,"Menu..... It was okay obviously, but if you ha...",0
...,...,...
5100204,"Good Story, Great Series. The Anita Blake seri...",0
5100205,One Star. Not Recommended,0
5100206,"My once and future desert island book:. \\""The...",1
5100207,A well-written story about a woman samurai. Re...,1


In [16]:
df_remaining['review_sentiments'].value_counts()

1    3628546
0     471663
Name: review_sentiments, dtype: int64

In [17]:
from sklearn.utils import shuffle
df_stratified = shuffle(df_stratified)
df_remaining = shuffle(df_remaining)

In [18]:
df_stratified

,review,review_sentiments
3837169,Disappointing treatment of a great album. This...,0
2882480,Good read. I have read all the books in this s...,1
4587996,"NOPE.. Just my opinion obviously, but every ti...",0
196423,"Passing gas was the highlight, and I HATED th...",0
94421,not believable **SPOILERS**. I really like his...,0
...,...,...
4996769,"Yuck!. Cardboard characters, stilted dialogue,...",0
1159844,Amazing book....... This was a rough story to ...,1
3565922,Great read. What a great story and to learn is...,1
5053743,tiresome tale. the purpose of the book seems t...,0


In [19]:
df_remaining

,review,review_sentiments
2009891,Poor. Poor writing skills of the author which ...,0
21510,Glad I finally got to this novel. To Fame's Pr...,1
4984599,Check it out!. I was pleasantly surprised by S...,1
1018345,Nice consolidation of business thought leaders...,1
2817065,Real page turner. I expected this to be Republ...,1
...,...,...
111832,Her Favorite Temptation. Leah Mathews has alwa...,0
4239286,A very fun read!. This story is the perfect le...,1
1399597,"A real page turner ""Who Dunnit?"" set in the de...",1
4119909,"12 PRECIOUS ANECDOTES FROM LIFE: A pleasant, a...",1


In [20]:
df_stratified.to_csv('../data/amazon_large_stratified.csv')

In [21]:
df_remaining.to_csv('../data/amazon_large_remaining.csv')